In [1]:
# you get all pretrained model name here
# https://huggingface.co/transformers/pretrained_models.html
import pandas as pd
import os
import torch
import transformers
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
# from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, EarlyStoppingCallback
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [2]:
# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
# model_name = "bert-base-uncased"
model_name = "distilbert-base-uncased"
model_dir = 'model_82'
# max sequence length for each document/sentence sample
max_length = 512

In [3]:
# tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [4]:
def read_passages(path_data, test_size=0.2):
    df = pd.read_csv(path_data)

    documents = df['processed'].to_list()
    labels_str = df['target'].to_list()
    
    samples = documents
    
    labels_list = sorted(list(set(labels_str)))
    labels_all = {l:idx for idx, l in enumerate(labels_list)}
    labels = [labels_all[label_str] for label_str in labels_str]

    train_samples, rest_samples, train_labels, rest_labels = train_test_split(samples, labels, stratify=labels, test_size=test_size)
    eval_samples, test_samples, eval_labels, test_labels = train_test_split(rest_samples, rest_labels, stratify=rest_labels, test_size=0.5)
        
    return train_samples, test_samples, eval_samples, train_labels, test_labels, eval_labels, labels_list
# end

In [5]:
# call the function
dir_data = 'datasource'
name_data_file = 'goscv_20221207_modelperformance_1.csv'
path_data_relative = os.path.join(dir_data, name_data_file)
train_samples, test_samples, valid_samples, train_labels, test_labels, valid_labels, target_names = read_passages(path_data_relative)

In [6]:
# train_samples[837]

In [7]:
train_encodings = tokenizer.batch_encode_plus(train_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
valid_encodings = tokenizer.batch_encode_plus(valid_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')
test_encodings = tokenizer.batch_encode_plus(test_samples, truncation=True, padding=True, max_length=max_length, return_tensors='pt')

In [8]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = SimpleDataset(train_encodings, train_labels)
valid_dataset = SimpleDataset(valid_encodings, valid_labels)
test_dataset = SimpleDataset(test_encodings, test_labels)

In [10]:
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names))
if os.path.exists(model_dir) and len(os.listdir(model_dir)) > 0:
    print('load model from local')
    model_info = model_dir
else:
    print('load model from official')
    model_info = model_name
    
model = DistilBertForSequenceClassification.from_pretrained(model_info, num_labels=len(target_names))

load model from official


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids.reshape(-1)
    # pred = np.argmax(pred, axis=1)
    preds = pred.predictions.argmax(-1).reshape(-1)

    # print('labels: {}'.format(labels))
    # print('pred: {}'.format(preds))
    accuracy = accuracy_score(y_true=labels, y_pred=preds)
    precision = precision_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    recall = recall_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')
    f1 = f1_score(y_true=labels, y_pred=preds, zero_division=1, average='macro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [12]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=2,  # batch size per device during training
    per_device_eval_batch_size=2,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=1,               # log & save weights each logging_steps
    evaluation_strategy="epoch",     # evaluate each `logging_steps`
    learning_rate=2e-5,
    save_strategy='epoch',
    save_total_limit=20,
    metric_for_best_model='f1'
)

In [13]:
# trainer = Trainer(
#     model=model,                         # the instantiated Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     compute_metrics=compute_metrics,     # the callback that computes metrics of interest
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
# )

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

In [14]:
trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 869
  Num Epochs = 20
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 8700
  Number of trainable parameters = 66958855
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.091500,0.643007,0.816514,0.844927,0.801398,0.811527
2,0.004400,0.401773,0.908257,0.958428,0.911712,0.930565
3,0.010100,0.344742,0.926606,0.934524,0.924995,0.927666
4,0.000900,0.400706,0.917431,0.947166,0.917766,0.929404
5,0.001200,0.275224,0.935780,0.938363,0.932225,0.934587
6,0.000800,0.406655,0.917431,0.928571,0.927275,0.925307
7,0.000300,0.403360,0.944954,0.939851,0.950082,0.944534
8,0.000400,0.374273,0.935780,0.943634,0.940558,0.941907
9,0.000500,0.503748,0.917431,0.918776,0.937069,0.926299
10,0.000100,0.407782,0.926606,0.918015,0.940828,0.926499


***** Running Evaluation *****
  Num examples = 109
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-435
Configuration saved in ./results/checkpoint-435/config.json
Model weights saved in ./results/checkpoint-435/pytorch_model.bin
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 109
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-870
Configuration saved in ./results/checkpoint-870/config.json
Model weights saved in ./results/checkpoint-870/pytorch_model.bin
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torc

TrainOutput(global_step=8700, training_loss=0.15887995870555882, metrics={'train_runtime': 591.1708, 'train_samples_per_second': 29.399, 'train_steps_per_second': 14.717, 'total_flos': 2302488678420480.0, 'train_loss': 0.15887995870555882, 'epoch': 20.0})

In [15]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 109
  Batch size = 2
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


{'eval_loss': 0.4191562533378601,
 'eval_accuracy': 0.944954128440367,
 'eval_precision': 0.9500819356082514,
 'eval_recall': 0.9500819356082514,
 'eval_f1': 0.9500819356082514,
 'eval_runtime': 0.7498,
 'eval_samples_per_second': 145.373,
 'eval_steps_per_second': 73.353,
 'epoch': 20.0}

In [16]:
model.save_pretrained(model_dir)

Configuration saved in model_82/config.json
Model weights saved in model_82/pytorch_model.bin


In [18]:
import json
os.makedirs(model_dir, exist_ok=True)
with open(f'{model_dir}/labels.json', 'w+') as file:
    file.write(json.dumps(target_names))

In [20]:
trainer2 = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=train_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)
trainer2.evaluate()

***** Running Evaluation *****
  Num examples = 869
  Batch size = 2
<ipython-input-8-869644865237>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}


{'eval_loss': 0.024549590423703194,
 'eval_accuracy': 0.9942462600690449,
 'eval_precision': 0.9968707169720294,
 'eval_recall': 0.9966678604956561,
 'eval_f1': 0.9967580234877758,
 'eval_runtime': 5.8293,
 'eval_samples_per_second': 149.075,
 'eval_steps_per_second': 74.623}